# Laboratorio 1 – Detección de Phishing
## Parte 2: Implementación de Modelos de Machine Learning

**Universidad del Valle de Guatemala**  
**CC3094 – Security Data Science | Semestre I - 2026**  
**Autores:** Fabiola Contreras (22787), María José Villafuerte (22129)  
**Docente:** Jorge Yass | **Sección:** 11

---

Este notebook implementa dos modelos de clasificación (**Random Forest** y **Logistic Regression**) para detectar URLs de phishing, utilizando las 7 características seleccionadas en la fase de ingeniería de características.

In [ ]:
# 0. Importación de librerías
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score,
    f1_score, accuracy_score, roc_curve, auc
)

# Configuración de visualización
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")
np.random.seed(42)

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 1. Carga y Preparación del Dataset

Se carga el dataset preprocesado (normalizado con StandardScaler) y se filtran las **7 características seleccionadas**:

| Característica | Justificación |
|---|---|
| `shannon_entropy` | Mide la aleatoriedad de caracteres en la URL; phishing presenta mayor entropía |
| `digit_ratio` | Proporción de dígitos; URLs con muchos números son sospechosas |
| `sensitive_words` | Presencia de palabras como "login", "verify", "account" |
| `url_length` | Longitud total; phishing tiende a usar URLs más largas para ocultar el dominio real |
| `domain_length` | Longitud del dominio; dominios falsos suelen ser más largos |
| `num_params` | Cantidad de parámetros en query string; URLs complejas son sospechosas |
| `special_chars` | Conteo de caracteres especiales no alfanuméricos |

In [ ]:
# 1. Carga del dataset preprocesado (normalizado)

DATA_PATH = '../feature engineering/dataset_phishing_preprocessed.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dataset cargado desde: {DATA_PATH}")
except FileNotFoundError:
    print(f"Archivo no encontrado en {DATA_PATH}")
# ── Características seleccionadas ──
SELECTED_FEATURES = [
    'shannon_entropy', 'digit_ratio', 'sensitive_words',
    'url_length', 'domain_length', 'num_params', 'special_chars'
]

# Verificar columnas
missing = [c for c in SELECTED_FEATURES + ['status'] if c not in df.columns]
if missing:
    raise ValueError(f"Columnas faltantes: {missing}")

df_selected = df[SELECTED_FEATURES + ['status']].copy()

print(f"\nDimensiones del dataset: {df_selected.shape}")
print(f"\nDistribución de clases:")
print(f"   Legítimas (0): {(df_selected['status'] == 0).sum()}")
print(f"   Phishing  (1): {(df_selected['status'] == 1).sum()}")
print(f"\nPrimeras 5 observaciones:")
df_selected.head()

Dataset cargado desde: ../feature engineering/dataset_phishing_preprocessed.csv

Dimensiones del dataset: (11430, 8)

Distribución de clases:
   Legítimas (0): 5715
   Phishing  (1): 5715

Primeras 5 observaciones:


,shannon_entropy,digit_ratio,sensitive_words,url_length,domain_length,num_params,special_chars,status
0,-1.132160,-0.594666,-0.341452,-0.436245,-0.194860,-0.284531,-0.489972,0
1,0.570316,1.875919,-0.341452,0.287212,0.176271,-0.284531,-0.489972,1
2,1.578933,1.092760,3.677903,1.173447,2.681407,2.642731,1.487912,1
3,-2.045956,-0.594666,-0.341452,-0.779888,-0.937123,-0.284531,-0.819619,0
4,-0.260735,-0.594666,-0.341452,-0.110690,-0.565991,-0.284531,0.004499,0
